In [52]:
import numpy as np
import pandas as pd


In [ ]:
#create a string from positive reviews and find the 10 most frequent words
#create a string from negative reviews and find the 10 most frequent words

#those words will form the vector that will decide in which category a review should be stored in


#each review will be represented by a vector(list) of these words, in each position the vector will have 0 or 1 for "contained" or "not-contained" respectivel
#in each vector there will be an 11th position that will have 0 or 1 representing the category the review

In [124]:
from pathlib import Path


#DO THE SAME FOR THE NEGATIVE REVIEWS TO CHOOSE THE MOST COMMON WORDS BETWEEN ALL THE DATA 
#THEN FIND THE POSSIBILITY OF A WORD EXISTING IN POSITIVE AND THEN IN NEGATIVE REVIEWS, CALCULATE THE IG OF EACH WORD AND SORT THEM WITH DESCENDING IG VALUE


files_positive = Path('/Users/michail/Downloads/aclImdb/train/pos').glob('*.txt')
files_negative = Path('/Users/michail/Downloads/aclImdb/train/neg').glob('*.txt')

print(files)

total_pos=0
total_neg=0

text= ""
i=0
for file in files_positive:
    #print(file.name)
    
    text= text+file.read_text()  # the file is opened and closed
    total_pos += 1
    """
    i+=1
    if i==500:
        break
    """    

    
    
for file in files_negative:
    #print(file.name)
    
    total_neg+=1
    text= text+file.read_text()  
    

print("Total positive reviews: ",  total_pos)
print("Total negative reviews: ", total_neg)


<generator object Path.glob at 0x7fe194e727b0>
Total positive reviews:  12500
Total negative reviews:  12500


In [125]:
from collections import Counter
from nltk.stem import PorterStemmer
import math
from decimal import *

split_it = text.split()
bad_chars = [';', ':', "!", "*", " ", "(", ")", '\"', ".", ",", "/", ">", "<", " "]

for i in range(len(split_it)):
    for b in bad_chars:
        if b in split_it[i]:
            split_it[i] = split_it[i].replace(b, '')


Counters_found = Counter(split_it)
most_occur = Counters_found.most_common(1000)


"""
print("The table with the most common words: ")
print(most_occur)
"""


ps = PorterStemmer()
words=[]
for i in range(len(most_occur)):
    words.append(most_occur[i][0])
        


words_to_exclude = ['-', '--', ';', ':', "!", "*", " ", "(", ")", '\"', ".", 'it\'s', 'br', 'mr', 'there\'s', 'your', 'wasn\'t', 'Ms', 'can', 'do', 'were', 'how', 'get', 'will', 'also', 'been', 'some', 'into', 'because', 'about', 'out', 'me', 'up', 'down', 'my', 'mine', 'their', 'she', 'he\'s', 'the','you', 'an', 'his', 'him', 'her', 'or', 'was', 'have', 'has', 'had', 'in', 'i', 'he', 'we', 'they', 'their', 'theirs', 'which','what', 'where', 'be', 'they', 'has', 'so',  'by', 'who', 'that','this', 'those', 'your', 'these', 'on', 'there', 'and', 'to', 'a', 'it', 'its', 'for', 'if', 'then', 'is', 'at', 'are', 'of', 'no', 'as', 'but', 'with', 'there']    

words_final=[]
for i, w in enumerate(words):
    
    #print(" w is: " + w)
    
    for b in bad_chars:
        if b in words[i]:
            words[i]= words[i].replace(b,'')
            w=words[i]
        
    
    #print("Reading the word " + words[i])
    
    
    if w.lower() in words_to_exclude:
        #print('removing ' +w)
        #words.remove(w)
        continue

    words_final.append(words[i])        
        

    words_final= words_final[:len(words_final)-60]
        
        
c=1
for i in  words_final:
    print(c,'. '+i)
    c=c+1

In [126]:
def truncate_float(float_number, decimal_places):
    multiplier = 10 ** decimal_places
    return int(float_number * multiplier) / multiplier


In [168]:
def IG(w, total_pos, total_neg):
    total_reviews= total_pos + total_neg
    
    files_positive = Path('/Users/michail/Downloads/aclImdb/train/pos').glob('*.txt')
    files_negative = Path('/Users/michail/Downloads/aclImdb/train/neg').glob('*.txt')
    
    
    P1= (total_pos/total_reviews)  #probability of positive reviews
    P0= (total_neg/total_reviews)  #probability of negative reviews
    
    entropy= -P1*math.log2(P1)-P0*math.log2(P0)
    #print("Total entropy: ", entropy)
    

    
    print ("Entropy is: ", entropy)
    
    #Probabilities of C=pos and w=1 & C=neg and w=1
    pos_counter=0
    neg_counter=0 
    total_counter=0
    
    
    for f in files_positive:
        if w in f.read_text():
            #print("it's in positive: ", w)
            pos_counter+=1
            total_counter+=1
      
    for f in files_negative:
        if w in f.read_text():
            #print("it's in negative: ", w)
            neg_counter+=1
            total_counter+=1
    
    
    #Probability of w=1 and w=0 
    P_w1 = total_counter/total_reviews
    print("Probability of ", w, "=1: ", P_w1)
    P_w0 = (total_reviews- total_counter) / total_reviews
    print("Probability of ", w, "=0: ", P_w0)

        
   
    #Probability of a C=pos and C=neg when w=1
    P1_w1= pos_counter/total_counter 
    print("probability positive reviews with ", w,"=1: ", P1_w1)
    P0_w1= neg_counter/total_counter
    print("probability negative reviews with ", w,"=1: ", P0_w1)
    
    
    
    
    #Probability of a C=pos and C=neg when w=0
    if P_w0!=0:
        P1_w0 = (total_pos-pos_counter)/(total_reviews- total_counter)
        print("probability positive reviews with ", w,"=0: ", P1_w0)
        P0_w0 = (total_neg-neg_counter)/(total_reviews - total_counter)
        print("probability positive reviews with ", w,"=0: ", P0_w0)
    else:
        P1_w0 = 0
        P0_w0 = 0
    
    
    entropy_w1= (0 if P1_w1==0 else -P1_w1*math.log2(P1_w1)) - (0 if P0_w1==0 else P0_w1*math.log2(P0_w1))
    print("Entropy of ", w, "=1: ", entropy_w1)
    entropy_w0= (0 if P1_w0==0 else -P1_w0*math.log2(P1_w0)) - (0 if P0_w0==0 else P0_w0*math.log2(P0_w0))
    print("Entropy of ", w, "=0: ", entropy_w0)

    
    if entropy!=1.0:
        ig= truncate_float(entropy, 4) - truncate_float((P_w1*entropy_w1 + P_w0*entropy_w0), 4)
        print("Truncated P_w1: ", truncate_float(P_w1*entropy_w1, 4))
        print("Truncated P_w0: ", truncate_float(P_w0*entropy_w0, 4))
        print("Truncated entropy: ", truncate_float(entropy,4))
    
    else:
        getcontext().prec = 6
        ig= Decimal(1) - Decimal((P_w1*entropy_w1 + P_w0*entropy_w0))
        print("Decimal P_w1: ", Decimal(P_w1*entropy_w1))
        print("Decimal P_w0: ", Decimal(P_w0*entropy_w0))
        print("Decimal entropy: ", Decimal(entropy))                        
        
        
      
    
    #print("IG type: ", type(ig))
    
    
    print ("IG type is: (", entropy, ") - (", P_w1, "*", entropy_w1, " + ",P_w0, "*", entropy_w0,")")
    
    
    print("IG of ",w , "is: ", ig)
    
    return ig
    
    
    

In [169]:
ig= IG("see", total_pos, total_neg)
print("IG IS: ", ig)

Entropy is:  1.0
Probability of  see =1:  0.58956
Probability of  see =0:  0.41044
probability positive reviews with  see =1:  0.4963701743673248
probability negative reviews with  see =1:  0.5036298256326752
probability positive reviews with  see =0:  0.5052139167722445
probability positive reviews with  see =0:  0.4947860832277556
Entropy of  see =1:  0.999961982822041
Entropy of  see =0:  0.9999215594564272
Decimal P_w1:  0.58953758659256239038626290493994019925594329833984375
Decimal P_w0:  0.410407804863296032937824975306284613907337188720703125
Decimal entropy:  1
IG type:  <class 'decimal.Decimal'>
IG type is: ( 1.0 ) - ( 0.58956 * 0.999961982822041  +  0.41044 * 0.9999215594564272 )
IG of  see is:  0.0000546085
IG IS:  0.0000546085


In [173]:
s= ig > 0.000055
print(s)

False


In [155]:
#USE IG IN EVERY WORD OF THE ARRAY

words_IG=[]

for w in words_final:
    words_IG.append(IG(w, total_pos, total_neg))